![image info](https://raw.githubusercontent.com/albahnsen/MIAD_ML_and_NLP/main/images/banner_1.png)

# Proyecto 2 - Clasificación de género de películas

El propósito de este proyecto es que puedan poner en práctica, en sus respectivos grupos de trabajo, sus conocimientos sobre técnicas de preprocesamiento, modelos predictivos de NLP, y la disponibilización de modelos. Para su desarrollo tengan en cuenta las instrucciones dadas en la "Guía del proyecto 2: Clasificación de género de películas"

**Entrega**: La entrega del proyecto deberán realizarla durante la semana 8. Sin embargo, es importante que avancen en la semana 7 en el modelado del problema y en parte del informe, tal y como se les indicó en la guía.

Para hacer la entrega, deberán adjuntar el informe autocontenido en PDF a la actividad de entrega del proyecto que encontrarán en la semana 8, y subir el archivo de predicciones a la [competencia de Kaggle](https://www.kaggle.com/t/2c54d005f76747fe83f77fbf8b3ec232).

## Datos para la predicción de género en películas

![image info](https://raw.githubusercontent.com/albahnsen/MIAD_ML_and_NLP/main/images/moviegenre.png)

En este proyecto se usará un conjunto de datos de géneros de películas. Cada observación contiene el título de una película, su año de lanzamiento, la sinopsis o plot de la película (resumen de la trama) y los géneros a los que pertenece (una película puede pertenercer a más de un género). Por ejemplo:
- Título: 'How to Be a Serial Killer'
- Plot: 'A serial killer decides to teach the secrets of his satisfying career to a video store clerk.'
- Generos: 'Comedy', 'Crime', 'Horror'

La idea es que usen estos datos para predecir la probabilidad de que una película pertenezca, dada la sinopsis, a cada uno de los géneros.

## MODELADO:
 En el notebook, realicen el preprocesamiento de los datos que consideren pertinente, así como la selección, la calibración y el entrenamiento del modelo predictivo con los datos de entrenamiento.

In [2]:
import warnings
warnings.filterwarnings('ignore')
# Importación librerías
import pandas as pd
import os
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.multiclass import OneVsRestClassifier
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier
from sklearn.metrics import r2_score, roc_auc_score
from sklearn.model_selection import train_test_split
import joblib

# Carga de datos de archivo .csv
dataTraining = pd.read_csv('https://github.com/albahnsen/MIAD_ML_and_NLP/raw/main/datasets/dataTraining.zip', encoding='UTF-8', index_col=0)
dataTesting = pd.read_csv('https://github.com/albahnsen/MIAD_ML_and_NLP/raw/main/datasets/dataTesting.zip', encoding='UTF-8', index_col=0)

# Visualización datos de entrenamiento
dataTraining.head()

# Visualización datos de test
# dataTesting.head()

,year,title,plot,genres,rating
3107,2003,Most,most is the story of a single father who takes...,"['Short', 'Drama']",8.0
900,2008,How to Be a Serial Killer,a serial killer decides to teach the secrets o...,"['Comedy', 'Crime', 'Horror']",5.6
6724,1941,A Woman's Face,"in sweden , a female blackmailer with a disfi...","['Drama', 'Film-Noir', 'Thriller']",7.2
4704,1954,Executive Suite,"in a friday afternoon in new york , the presi...",['Drama'],7.4
2582,1990,Narrow Margin,"in los angeles , the editor of a publishing h...","['Action', 'Crime', 'Thriller']",6.6


In [3]:
import pandas as pd

# Carregar os dados (substitua pelo caminho real do seu arquivo)
dataTraining = pd.read_csv('https://github.com/albahnsen/MIAD_ML_and_NLP/raw/main/datasets/dataTraining.zip', encoding='UTF-8', index_col=0)

# Função para obter todos os gêneros únicos
def obter_generos_unicos(df, coluna):
    generos_unicos = set()
    for generos in df[coluna]:
        generos_lista = eval(generos)  # Converte a string da lista de volta para uma lista
        for genero in generos_lista:
            generos_unicos.add(genero)
    return generos_unicos

# Obter todos os gêneros únicos
todos_generos = obter_generos_unicos(dataTraining, 'genres')
print("Todos os gêneros únicos na coluna 'genres':")
print(todos_generos)

Todos os gêneros únicos na coluna 'genres':
{'Documentary', 'Crime', 'Sport', 'Romance', 'Music', 'Fantasy', 'Musical', 'Biography', 'Family', 'Comedy', 'News', 'Short', 'Mystery', 'Horror', 'Film-Noir', 'Adventure', 'Drama', 'Western', 'Thriller', 'War', 'Animation', 'History', 'Action', 'Sci-Fi'}


### PREPROCESAMIENTO

In [4]:
#mejor:PREPROCESAMIENTO3:n-gramas,TF-IDF Vectorization Y Lematización
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from sklearn.preprocessing import MultiLabelBinarizer
import string
import nltk

nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('omw-1.4')

# Preprocesamiento de Texto
stop_words = set(stopwords.words('english'))
lemmatizer = WordNetLemmatizer()

def preprocess_text(text):
    # Convertir todo a minúsculas
    text = text.lower()
    # Tokenizar el texto
    tokens = word_tokenize(text)
    # Eliminar stopwords, puntuaciones y números
    tokens = [word for word in tokens if word not in stop_words and word not in string.punctuation and not word.isdigit()]
    # Lematización
    tokens = [lemmatizer.lemmatize(word) for word in tokens]
    # Unir los tokens nuevamente en un string
    return ' '.join(tokens)

# Aplicar preprocesamiento al texto
dataTraining['plot_processed'] = dataTraining['plot'].apply(preprocess_text)

# Vectorización del Texto usando TF-IDF con n-gramas
#vectorizer = TfidfVectorizer(max_features=7800) #mejor modelo regresion logistica 0.88, SVR
vectorizer = TfidfVectorizer(max_features=5000, ngram_range=(1, 2)) #para el stacking,redes 0.86

X = vectorizer.fit_transform(dataTraining['plot_processed'])
y = dataTraining['genres']

# Binarizar la variable objetivo
mlb = MultiLabelBinarizer()
y = mlb.fit_transform(y.apply(eval))

# División de los datos en entrenamiento y prueba
X_train4, X_test4, y_train4, y_test4 = train_test_split(X, y, test_size=0.2, random_state=42)

[nltk_data] Downloading package punkt to /home/felipe/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /home/felipe/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /home/felipe/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /home/felipe/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


In [6]:
# Crear tfidf_vectorizer.pkl
joblib.dump(vectorizer, 'tfidf_vectorizer.pkl')

['tfidf_vectorizer.pkl']

In [7]:
joblib.dump(mlb, 'mlb.pkl')

['mlb.pkl']

### MODELADO CON PREPROCESAMIENTO

In [19]:
#MODELO PROFE
#Definición y entrenamiento
clf4 = OneVsRestClassifier(RandomForestClassifier(n_jobs=-1, n_estimators=100, max_depth=10, random_state=42))
clf4.fit(X_train4, y_train4)
# Predicción del modelo de clasificación
y_pred_rf4= clf4.predict_proba(X_test4)
# Impresión del desempeño del modelo
roc_auc_score(y_test4, y_pred_rf4, average='macro')
#0.8255629781604181

0.8255629781604181

In [3]:
#MEJOR:Regresión Logística Multiclase
from sklearn.linear_model import LogisticRegression
# Crear el modelo de regresión logística multiclase
logistic_reg4 = OneVsRestClassifier(LogisticRegression(max_iter=1000, random_state=42))
logistic_reg4.fit(X_train4, y_train4)
y_pred_logis4 = logistic_reg4.predict_proba(X_test4)
roc_auc_score(y_test4, y_pred_logis4, average='macro')
#0.8891961604881898

0.8859398196321558

In [20]:
#Naive Bayes
from sklearn.naive_bayes import MultinomialNB
nb_classifier = OneVsRestClassifier(MultinomialNB())
nb_classifier.fit(X_train4, y_train4)
y_pred_NB2 = nb_classifier.predict(X_test4)
roc_auc_score(y_test4, y_pred_NB2, average='macro')
#0.522705170276073

0.522705170276073

In [17]:
#LightGBM
import lightgbm as lgb
lgbm = lgb.LGBMClassifier(random_state=42,n_jobs=-1)
classifier = OneVsRestClassifier(lgbm)
classifier.fit(X_train4, y_train4)
y_pred_ligth = classifier.predict_proba(X_test4)
auc_score = roc_auc_score(y_test4, y_pred_ligth, average='macro')
print("AUC Score:", auc_score)
#AUC Score: 0.8505648915346008

[LightGBM] [Info] Number of positive: 1039, number of negative: 5277
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.098004 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 90340
[LightGBM] [Info] Number of data points in the train set: 6316, number of used features: 3132
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.164503 -> initscore=-1.625099
[LightGBM] [Info] Start training from score -1.625099
[LightGBM] [Info] Number of positive: 816, number of negative: 5500
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.193805 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 90340
[LightGBM] [Info] Number of data points in the train set: 6316, number of used features: 3132
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.129196 -> initscore=-1.908089
[LightGBM] [Info] Start training from score -1.908089
[LightGBM] [I

In [ ]:
#CatBoost nunca termino
from catboost import CatBoostClassifier
catboost_model = CatBoostClassifier(verbose=0, random_seed=42)
ovr_catboost = OneVsRestClassifier(catboost_model)
ovr_catboost.fit(X_train4, y_train4)
y_pred_catbost = ovr_catboost.predict_proba(X_test4)
auc_score = roc_auc_score(y_test4, y_pred_catbost, average='macro')
print("AUC Score:", auc_score)

In [91]:
#CALIBRACION#1 con RandomizedSearchCV no se uso gride por lo pesado que puede ser 
from sklearn.model_selection import RandomizedSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.multiclass import OneVsRestClassifier

# Crear el modelo base de regresión logística
base_model = LogisticRegression(max_iter=1000, random_state=42)

# Crear el clasificador OneVsRest con el modelo base
classifier = OneVsRestClassifier(base_model)

# Definir los parámetros a buscar
param_dist = {'estimator__C': [0.001, 0.01, 0.1, 1, 10, 100]}
random_search = RandomizedSearchCV(estimator=classifier,
                                   param_distributions=param_dist,
                                   n_iter=10,
                                   scoring='roc_auc',
                                   cv=5,
                                   verbose=2,
                                   n_jobs=-1)


# Realizar la búsqueda de hiperparámetros
random_search.fit(X_train4, y_train4)

# Obtener el mejor modelo y sus hiperparámetros
best_model = random_search.best_estimator_
best_params = random_search.best_params_

# Imprimir los mejores parámetros y el mejor puntaje
print("Mejores parámetros encontrados:", best_params)
print("Mejor puntaje (ROC AUC):", random_search.best_score_)


# Entrenar el modelo con los datos de entrenamiento
best_model.fit(X_train4, y_train4)

# Predecir las probabilidades en el conjunto de prueba
y_pred_proba = best_model.predict_proba(X_test4)

# Calcular el AUC
auc = roc_auc_score(y_test4, y_pred_proba, average='macro')
print("AUC:", auc)
#AUC-ROC Score: 0.8580425171177647

Fitting 5 folds for each of 6 candidates, totalling 30 fits
Mejores parámetros encontrados: {'estimator__C': 0.001}
Mejor puntaje (ROC AUC): nan
AUC: 0.8825829497048616


In [35]:
#STACKING
# Definir modelos base
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.multiclass import OneVsRestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import MultinomialNB
from sklearn.ensemble import RandomForestClassifier, StackingClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score


# Definición de los modelos base
model1 = ('logistic_reg', LogisticRegression(max_iter=1000, random_state=42))
model3 = ('random_forest', RandomForestClassifier(n_jobs=-1, n_estimators=50, max_depth=10, random_state=42))

# Definición del meta-modelo
meta_model = LogisticRegression(max_iter=1000, random_state=42)

# Definición del Stacking Classifier
stacking_model = StackingClassifier(
    estimators=[model1, model3],
    final_estimator=meta_model,
    cv=3,  
    n_jobs=1)

# Usar MultiOutputClassifier para manejar múltiples salidas
multi_output_stacking = MultiOutputClassifier(stacking_model, n_jobs=1)

# Entrenamiento del Stacking Classifier
multi_output_stacking.fit(X_train4, y_train4)

# Predicción del modelo de ensamblaje
y_pred_stacking = multi_output_stacking.predict_proba(X_test4)

# Evaluación del desempeño del modelo
auc_score = roc_auc_score(y_test4, np.hstack([y[:, 1][:, np.newaxis] for y in y_pred_stacking]), average='macro')
print("AUC Score del Stacking Classifier:", auc_score)
#0.8796844834549841

AUC Score del Stacking Classifier: 0.8796844834549841


In [26]:
#STACKING2
# Definir modelos base
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.multiclass import OneVsRestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import MultinomialNB
from sklearn.ensemble import RandomForestClassifier, StackingClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score
from sklearn.multioutput import MultiOutputClassifier


# Definición de los modelos base
model1 = ('logistic_reg', LogisticRegression(max_iter=1000, random_state=42))
model2=('SV',SVC(kernel='linear', probability=True, random_state=42))
# Definición del meta-modelo
meta_model = LogisticRegression(max_iter=1000, random_state=42)

# Definición del Stacking Classifier
stacking_model = StackingClassifier(
    estimators=[model1,model2],
    final_estimator=meta_model,
    cv=3,  
    n_jobs=1)

# Usar MultiOutputClassifier para manejar múltiples salidas
multi_output_stacking2 = MultiOutputClassifier(stacking_model, n_jobs=1)

# Entrenamiento del Stacking Classifier
multi_output_stacking2.fit(X_train4, y_train4)

# Predicción del modelo de ensamblaje
y_pred_stacking = multi_output_stacking2.predict_proba(X_test4)

# Evaluación del desempeño del modelo
auc_score = roc_auc_score(y_test4, np.hstack([y[:, 1][:, np.newaxis] for y in y_pred_stacking]), average='macro')
print("AUC Score del Stacking Classifier:", auc_score)
#AUC Score del Stacking Classifier: 0.8485437202542346

In [10]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from scikeras.wrappers import KerasClassifier
from sklearn.model_selection import RandomizedSearchCV
from sklearn.metrics import roc_auc_score
import numpy as np

# Definir una función para crear el modelo
def create_model(neurons=128, learning_rate=0.001, dropout_rate=0.5):
    model = Sequential()
    model.add(Dense(neurons, input_dim=X_train4.shape[1], activation='relu'))
    model.add(Dropout(dropout_rate))
    model.add(Dense(y_train4.shape[1], activation='sigmoid'))
    optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate)
    model.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['accuracy'])
    return model
# Crear el KerasClassifier usando scikeras
model = KerasClassifier(model=create_model, verbose=0)
# Definir el espacio de búsqueda de hiperparámetros
param_dist = {
    'model__neurons': [64, 128, 256],
    'model__learning_rate': [0.001, 0.01, 0.1],
    'model__dropout_rate': [0.3, 0.5, 0.7],
    'batch_size': [32, 64, 128],
    'epochs': [10, 20]
}
# Realizar la búsqueda de hiperparámetros
random_search = RandomizedSearchCV(estimator=model, param_distributions=param_dist, n_iter=10, scoring='roc_auc', cv=3, verbose=2, n_jobs=-1)
# Entrenar el modelo usando RandomizedSearchCV
random_search.fit(X_train4.toarray(), y_train4)
# Obtener el mejor modelo y sus hiperparámetros
best_model = random_search.best_estimator_
best_params = random_search.best_params_
# Imprimir los mejores parámetros y el mejor puntaje
print("Mejores parámetros encontrados:", best_params)
print("Mejor puntaje (ROC AUC):", random_search.best_score_)

Fitting 3 folds for each of 10 candidates, totalling 30 fits
Mejores parámetros encontrados: {'model__neurons': 256, 'model__learning_rate': 0.001, 'model__dropout_rate': 0.5, 'epochs': 10, 'batch_size': 32}
Mejor puntaje (ROC AUC): 0.8687475581760081


In [13]:
from sklearn.model_selection import GridSearchCV

# Definir el espacio de búsqueda de hiperparámetros (menos combinaciones para aligerar el proceso)
param_grid = {
    'model__neurons': [64, 128],
    'model__learning_rate': [0.001, 0.01],
    'model__dropout_rate': [0.3, 0.5],
    'batch_size': [32, 64],
    'epochs': [10, 20]
}

# Realizar la búsqueda de hiperparámetros
grid_search = GridSearchCV(estimator=model, param_grid=param_grid, scoring='roc_auc', cv=3, verbose=2, n_jobs=-1)

# Entrenar el modelo usando GridSearchCV
grid_search.fit(X_train4.toarray(), y_train4)

# Obtener el mejor modelo y sus hiperparámetros
best_model = grid_search.best_estimator_
best_params = grid_search.best_params_

# Imprimir los mejores parámetros y el mejor puntaje
print("Mejores parámetros encontrados:", best_params)
print("Mejor puntaje (ROC AUC):", grid_search.best_score_)

# Evaluación del mejor modelo en el conjunto de prueba
y_pred_RED1_best = best_model.predict(X_test4.toarray())
auc_score = roc_auc_score(y_test4, y_pred_RED1_best, average='macro')
print("AUC Score del mejor modelo de red neuronal:", auc_score)

Fitting 3 folds for each of 32 candidates, totalling 96 fits
Mejores parámetros encontrados: {'batch_size': 32, 'epochs': 10, 'model__dropout_rate': 0.3, 'model__learning_rate': 0.001, 'model__neurons': 128}
Mejor puntaje (ROC AUC): 0.8641089465419985
AUC Score del mejor modelo de red neuronal: 0.6230654726763062


In [61]:
#RED2: PARA ASEGURAR Q DE MEJOR QUE LA RED ORIGINAL 
from tensorflow.keras.optimizers import Adam, RMSprop
from tensorflow.keras.regularizers import l2
model = Sequential()
model.add(Dense(256, input_dim=X_train4.shape[1], activation='relu', kernel_regularizer=l2(0.001)))
model.add(Dropout(0.5))
model.add(Dense(128, activation='relu', kernel_regularizer=l2(0.001)))
model.add(Dropout(0.5))
model.add(Dense(y_train4.shape[1], activation='sigmoid'))
# Compilar el modelo
model.compile(optimizer=RMSprop(learning_rate=0.0005), loss='binary_crossentropy', metrics=['accuracy'])
# Early stopping
early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)
# Entrenar el modelo
history = model.fit(X_train4.toarray(), y_train4, epochs=50, batch_size=64, validation_split=0.2, verbose=1, callbacks=[early_stopping])
# Evaluación del modelo
y_pred_RED3 = model.predict(X_test4.toarray())
auc_score = roc_auc_score(y_test4, y_pred_RED3, average='macro')
print("AUC Score de la red neuronal:", auc_score)
#0.611088377770746

Epoch 1/50
79/79 [==============================] - 2s 21ms/step - loss: 1.1093 - accuracy: 0.1663 - val_loss: 0.8154 - val_accuracy: 0.2104
Epoch 2/50
79/79 [==============================] - 1s 18ms/step - loss: 0.6775 - accuracy: 0.1997 - val_loss: 0.5682 - val_accuracy: 0.2104
Epoch 3/50
79/79 [==============================] - 2s 23ms/step - loss: 0.5243 - accuracy: 0.2031 - val_loss: 0.4626 - val_accuracy: 0.2104
Epoch 4/50
79/79 [==============================] - 2s 19ms/step - loss: 0.4391 - accuracy: 0.2065 - val_loss: 0.3982 - val_accuracy: 0.2104
Epoch 5/50
79/79 [==============================] - 2s 20ms/step - loss: 0.3852 - accuracy: 0.2043 - val_loss: 0.3586 - val_accuracy: 0.2104
Epoch 6/50
79/79 [==============================] - 2s 23ms/step - loss: 0.3531 - accuracy: 0.2057 - val_loss: 0.3341 - val_accuracy: 0.2104
Epoch 7/50
79/79 [==============================] - 2s 21ms/step - loss: 0.3323 - accuracy: 0.2126 - val_loss: 0.3187 - val_accuracy: 0.2104
Epoch 8/50
79

## Predicción conjunto de test para envío a Kaggle
En esta sección encontrarán el formato en el que deben guardar los resultados de la predicción para que puedan subirlos a la competencia en Kaggle.

In [ ]:
# transformación variables predictoras X del conjunto de test
X_test_dtm = vectorizer.transform(dataTesting['plot'])

cols = ['p_Action', 'p_Adventure', 'p_Animation', 'p_Biography', 'p_Comedy', 'p_Crime', 'p_Documentary', 'p_Drama', 'p_Family',
        'p_Fantasy', 'p_Film-Noir', 'p_History', 'p_Horror', 'p_Music', 'p_Musical', 'p_Mystery', 'p_News', 'p_Romance',
        'p_Sci-Fi', 'p_Short', 'p_Sport', 'p_Thriller', 'p_War', 'p_Western']

# Predicción del conjunto de test
y_pred_test_genres = logistic_reg4.predict_proba(X_test_dtm)
# Guardar predicciones en formato exigido en la competencia de kaggle
res = pd.DataFrame(y_pred_test_genres, index=dataTesting.index, columns=cols)
res.to_csv('pred_preprocesam5.csv', index_label='ID')
res.head()

In [4]:
#Exportando modelo
import sklearn
import joblib
print(sklearn.__version__)
joblib.dump(logistic_reg4, 'modelo.pkl', protocol=4)

1.4.2


['modelo.pkl']